In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
# !wget https://huggingface.co/datasets/mesolitica/vad-test-set/resolve/main/vad-test-63.pkl

In [3]:
import pickle
import malaya_speech
from sklearn import metrics

`pyaudio` is not available, `malaya_speech.streaming.pyaudio_vad.stream` is not able to use.


In [4]:
with open('vad-test-63.pkl', 'rb') as fopen:
    data = pickle.load(fopen)

In [5]:
data[0]['waveforms'].shape

(1008,)

In [6]:
model = malaya_speech.vad.deep_model(model = 'marblenet-factor3')

Downloading:   0%|          | 0.00/3.21M [00:00<?, ?B/s]

2023-02-14 13:25:45.005499: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-14 13:25:45.018577: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-02-14 13:25:45.018631: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: husein-MS-7D31
2023-02-14 13:25:45.018636: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: husein-MS-7D31
2023-02-14 13:25:45.018776: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.161.3
2023-02-14 13:25:45.018832: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported vers

In [7]:
from tqdm import tqdm

In [ ]:
predict_Y = [model(f['waveforms']) for f in tqdm(data)]

 58%|████████████████████▍              | 58281/100000 [01:33<01:19, 521.51it/s]

In [ ]:
from sklearn import metrics

print(
    metrics.classification_report(
        [f['targets'] for f in data], predict_Y,
        digits = 5
    )
)